# General Assembly DAT-28 Kaggle Competition

In [1]:
import pandas as pd
import numpy as np

In [2]:
ls

kaggle-ga.ipynb   submission2.csv   submission_1.csv  test.csv*         train.csv


In [3]:
df = pd.read_table('train.csv', sep=',', parse_dates=[1,3,13], nrows=100000)

In [4]:
df.head()

,Id,PostCreationDate,OwnerUserId,OwnerCreationDate,ReputationAtPostCreation,OwnerUndeletedAnswerCountAtPostTime,Title,BodyMarkdown,Tag1,Tag2,Tag3,Tag4,Tag5,PostClosedDate,OpenStatus
0,6458464,2011-06-23 17:53:27,663229,2011-03-16 20:19:28,210,0,Generating HTML,\t <form action=post ...>\r\r\n //...,html,c,NaN,NaN,NaN,NaT,open
1,4251761,2010-11-23 00:58:05,200477,2009-11-01 12:41:57,1715,13,Execute automake from eclipse autotools plugin,I recently installed the autotools plugin for ...,c,eclipse,autotools,NaN,NaN,NaT,open
2,10187289,2012-04-17 07:55:22,1338090,2012-04-17 07:25:28,1,0,wxPython TextCtrl auto detect file/folder path,I'm looking for a way that path (Window or Lin...,wxpython,textctrl,NaN,NaN,NaN,2012-04-18 12:04:40,not a real question
3,10812620,2012-05-30 08:27:16,348545,2010-05-23 23:06:20,4290,75,Benefits of avoiding .pyc files?,"Are there any benefits, performance or otherwi...",python,NaN,NaN,NaN,NaN,NaT,open
4,10824532,2012-05-30 21:28:41,1423020,2012-05-29 06:49:48,1,0,threads with readline,I have a huge file with hundreds of thousands ...,python,python-2.7,NaN,NaN,NaN,NaT,open


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100000 entries, 0 to 99999
Data columns (total 15 columns):
Id                                     100000 non-null int64
PostCreationDate                       100000 non-null datetime64[ns]
OwnerUserId                            100000 non-null int64
OwnerCreationDate                      100000 non-null datetime64[ns]
ReputationAtPostCreation               100000 non-null int64
OwnerUndeletedAnswerCountAtPostTime    100000 non-null int64
Title                                  100000 non-null object
BodyMarkdown                           100000 non-null object
Tag1                                   100000 non-null object
Tag2                                   84455 non-null object
Tag3                                   58840 non-null object
Tag4                                   30949 non-null object
Tag5                                   12142 non-null object
PostClosedDate                         2079 non-null datetime64[ns]
OpenStat

In [6]:
df['OpenStatus'].value_counts()

open                   97921
not a real question      927
off topic                528
not constructive         438
too localized            186
Name: OpenStatus, dtype: int64

In [7]:
df['ClosedStatus'] = df['OpenStatus'].apply(lambda x: 0 if x == 'open' else 1)

## DummyClassifier

In [65]:
from sklearn.dummy import DummyClassifier
from sklearn.cross_validation import train_test_split
from sklearn.metrics import log_loss, accuracy_score, f1_score, precision_recall_fscore_support, roc_auc_score

X_train, X_test, y_train, y_test = train_test_split(df['Id'].values, df['ClosedStatus'].values)

dumb = DummyClassifier(strategy='most_frequent') #stratified
dumb.fit(X_train, y_train)
y_dumb_class = dumb.predict(X_test[:,None])

print ('Log Loss:', log_loss(y_test, y_dumb_class))
print ('Accuracy Score:', accuracy_score(y_test, y_dumb_class))
print ('ROC Score:', roc_auc_score(y_test, y_dumb_class))

Log Loss: 0.708735691624
Accuracy Score: 0.97948
F1 Score: 0.5


## Clean Data & Feature Engineering

In [9]:
import re

def clean_data(df):

    def remove_tags_and_format(text):
        text = re.sub('<[^<]+?>', '', text)
        text = re.sub( '\s+', ' ', text).strip()
        return text

    df['BodyMarkdown_clean'] = df['BodyMarkdown'].apply(remove_tags_and_format)
    
    df['Tags'] = df[['Tag1', 'Tag2', 'Tag3', 'Tag4', 'Tag5']].fillna('').apply(lambda x: ' '.join(x), axis=1)
    
    # days between date account created and date question posted
    df['AccountCreationPostCreationDiff'] = np.abs(df['OwnerCreationDate'] - df['PostCreationDate']).dt.days
    
    df['Title_len'] = df['Title'].str.len()
    df['BodyMarkdown_len'] = df['Title'].str.len()
    df['Tags_len'] = df['Tags'].apply(lambda x: len(x.split()))
    
    return df

In [10]:
df = clean_data(df)

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100000 entries, 0 to 99999
Data columns (total 22 columns):
Id                                     100000 non-null int64
PostCreationDate                       100000 non-null datetime64[ns]
OwnerUserId                            100000 non-null int64
OwnerCreationDate                      100000 non-null datetime64[ns]
ReputationAtPostCreation               100000 non-null int64
OwnerUndeletedAnswerCountAtPostTime    100000 non-null int64
Title                                  100000 non-null object
BodyMarkdown                           100000 non-null object
Tag1                                   100000 non-null object
Tag2                                   84455 non-null object
Tag3                                   58840 non-null object
Tag4                                   30949 non-null object
Tag5                                   12142 non-null object
PostClosedDate                         2079 non-null datetime64[ns]
OpenStat

## Random Forest Classifier

In [12]:
from sklearn.ensemble import RandomForestClassifier

In [13]:
X = df[['ReputationAtPostCreation', 
        'OwnerUndeletedAnswerCountAtPostTime', 
        'AccountCreationPostCreationDiff',
        'Title_len',
        'BodyMarkdown_len',
        'Tags_len']]
y = df['ClosedStatus']

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [14]:
rfclf = RandomForestClassifier(n_estimators=400, n_jobs=-1)

In [15]:
rfclf.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=400, n_jobs=-1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [16]:
y_pred_rfclf = rfclf.predict_proba(X_test)

In [17]:
print ('Log Loss:', log_loss(y_test, y_pred_rfclf))
print ('ROC Score:', roc_auc_score(y_test, y_pred_rfclf))

Log Loss: 0.236627055062


## Naive Bayes Classifier on Title Classifier

In [18]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB

In [19]:
# define X and y
X = df['Title'].values

In [20]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [21]:
vect_title = TfidfVectorizer(stop_words='english')

In [22]:
# fit and transform X_train
X_train_dtm = vect_title.fit_transform(X_train)

In [23]:
# import/instantiate/fit
nb_title = MultinomialNB()
nb_title.fit(X_train_dtm, y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [24]:
# make class predictions
X_test_dtm = vect_title.transform(X_test)
y_pred_nb_title = nb_title.predict_proba(X_test_dtm)

In [25]:
print ('Log Loss:', log_loss(y_test, y_pred_nb_title))
print ('ROC Score:', roc_auc_score(y_test, y_pred_nb_title))

Log Loss: 0.126609592816


## Naive Bayes Classifier on Body Feature

In [26]:
X = df['BodyMarkdown_clean'].values

In [27]:
vect_body = TfidfVectorizer(stop_words='english')

In [28]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [29]:
# fit and transform X_train
X_train_dtm = vect_body.fit_transform(X_train)

In [30]:
nb_body = MultinomialNB()
nb_body.fit(X_train_dtm, y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [31]:
# make class predictions
X_test_dtm = vect_body.transform(X_test)
y_pred_nb_body = nb_body.predict_proba(X_test_dtm)

In [32]:
print ('Log Loss:', log_loss(y_test, y_pred_nb_body))
print ('ROC Score:', roc_auc_score(y_test, y_pred_nb_body))

Log Loss: 0.260638666952


## Naive Bayes Classifier on Tags Feature

In [33]:
X = df['Tags'].values

In [34]:
vect_tags = TfidfVectorizer()

In [35]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [36]:
# fit and transform X_train
X_train_dtm = vect_tags.fit_transform(X_train)

In [37]:
nb_tags = MultinomialNB()
nb_tags.fit(X_train_dtm, y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [38]:
# make class predictions
X_test_dtm = vect_tags.transform(X_test)
y_pred_nb_tags = nb_tags.predict_proba(X_test_dtm)

In [39]:
print ('Log Loss:', log_loss(y_test, y_pred_nb_tags))
print ('ROC Score:', roc_auc_score(y_test, y_pred_nb_tags))

Log Loss: 0.110849678695


## Combine Predicted Probabilites into DataFrame

In [40]:
y_pred_nb_title[:,1]

array([ 0.0014495 ,  0.00344133,  0.00365125, ...,  0.00072573,
        0.00382046,  0.00242485])

In [41]:
pred_values = np.array([y_pred_nb_title[:,1], y_pred_nb_body[:,1], y_pred_nb_tags[:,1], y_pred_rfclf[:,1]])

In [42]:
pred_values.shape

(4, 25000)

In [43]:
df_pred = pd.DataFrame(pred_values.T, columns=['NB_Title', 'NB_Body', 'NB_Tags', 'RF_OtherFeat'])

In [ ]:
df_pred['y_test'] = y_test.reset_index(drop=True)

In [ ]:
df_pred.head(10)

## Average of Predicted Probabilities of all the Classifiers

In [49]:
print ('Log Loss:', log_loss(y_test, df_pred.max(axis=1).values))

Log Loss: 0.10659121816


## Voting Classifier

In [ ]:
from sklearn import cross_validation
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import VotingClassifier, ExtraTreesClassifier
from sklearn.svm import SVC

clf1 = LogisticRegression(n_jobs=-1)
clf2 = RandomForestClassifier(n_jobs=-1)
clf3 = SVC(probability=True)

eclf = VotingClassifier(estimators=[('lr', clf1), ('rf', clf2), ('svc', clf3)], voting='hard')

# define X and y
X = df_pred[['NB_Title', 'NB_Body', 'NB_Tags', 'RF_OtherFeat']].values
y = df_pred['y_test'].values

eclf.fit(X, y)

for clf, label in zip([clf1, clf2, clf3, eclf], ['Logistic Regression', 'Random Forest', 'SVC', 'Ensemble']):
    scores = cross_validation.cross_val_score(clf, X, y, cv=5, scoring='precision')
    print("Log Loss: %0.3f (+/- %0.3f) [%s]" % (scores.mean(), scores.std(), label))

# Predicting on Test Data

In [50]:
df_test = pd.read_table('test.csv', sep=',', parse_dates=[1,3])

In [51]:
df_test = clean_data(df_test)

In [52]:
X = df_test[['ReputationAtPostCreation', 
            'OwnerUndeletedAnswerCountAtPostTime', 
            'AccountCreationPostCreationDiff',
            'Title_len',
            'BodyMarkdown_len',
            'Tags_len']]

y_pred_rfclf = rfclf.predict_proba(X)

In [53]:
X = df_test['Title'].values

X_dtm = vect_title.transform(X)
y_pred_nb_title = nb_title.predict_proba(X_dtm)

In [54]:
X = df_test['BodyMarkdown_clean'].values

X_dtm = vect_body.transform(X)
y_pred_nb_body = nb_body.predict_proba(X_dtm)

In [55]:
X = df_test['Tags'].values

X_dtm = vect_tags.transform(X)
y_pred_nb_tags = nb_tags.predict_proba(X_dtm)

In [57]:
pred_values = np.array([y_pred_nb_title[:,1], y_pred_nb_body[:,1], y_pred_nb_tags[:,1], y_pred_rfclf[:,1]])
#pred_values = np.array([y_pred_rfclf[:,1]])

In [58]:
df_pred = pd.DataFrame(pred_values.T, columns=['NB_Title', 'NB_Body', 'NB_Tags', 'RF_OtherFeat'])

In [59]:
df_pred.head()

,NB_Title,NB_Body,NB_Tags,RF_OtherFeat
0,0.000857,1.177126e-06,0.017288,0.007500
1,0.007300,4.775158e-09,0.007949,0.000695
2,0.001520,1.899335e-07,0.001881,0.025000
3,0.001717,6.671659e-05,0.026743,0.002500
4,0.000886,1.723562e-07,0.001023,0.062500


In [ ]:
#y_pred = eclf.predict(df_pred)

In [60]:
#df_test['Prediction'] = y_pred[:,1]
df_test['Prediction'] = df_pred.max(axis=1).values

In [61]:
df_test[['Id', 'Prediction']].head()

,Id,Prediction
0,11321133,0.017288
1,9881029,0.007949
2,6205599,0.025000
3,1787549,0.026743
4,8894724,0.062500


In [62]:
df_test[['Id', 'Prediction']].to_csv('submission3.csv', index=False)